### Geocoding Toronto Neighborhoods

First I install the needed packages. Beautifulsoup4 and lxml for webscrapping and geocoder for, well, geocoding.

In [1]:
import sys
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

In [29]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import geocoder
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors


In [3]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, "lxml")
html_table = soup.table

In [4]:
postal_codes = pd.read_html(str(html_table))[0]
postal_codes = postal_codes.loc[postal_codes.Borough != "Not assigned"]
postal_list = postal_codes.loc[:,"Postcode"].drop_duplicates().to_list()

I request the data from wikipedia and take the first table found in the page, check that it's indeed the Postcode data set, drop missing values and make a list with all the unique values of the postcodes.

In [5]:
list_of_df = list()
for postal_code in postal_list:
    df = postal_codes.loc[postal_codes.Postcode == postal_code]
    neighborhoods = ', '.join(df.loc[:,"Neighbourhood"].to_list())
    df_2 = df.loc[:,["Postcode","Borough"]].drop_duplicates()
    df_2["Neighborhoods"] = neighborhoods
    list_of_df.append(df_2)
postal_codes = pd.concat(list_of_df)

To retrieve the data in the desired manner, I loop through the postcode list. For each value, I subset the original dataframe to get the name of the Borouh and a list of nieghborhoods, and then convert this list of neoghborhoods to a comma separated string and then assign it to the Neighborhood column. After doing this to everty postode, I concat all the data into a single data frame.

In [6]:
postal_codes.sample(20)

,Postcode,Borough,Neighborhoods
261,M4Y,Downtown Toronto,Church and Wellesley
93,M2K,North York,Bayview Village
13,M3B,North York,Don Mills North
41,M1E,Scarborough,"Guildwood, Morningside, West Hill"
14,M4B,East York,"Woodbine Gardens, Parkview Hill"
75,M1J,Scarborough,Scarborough Village
127,M3M,North York,Downsview Central
116,M6L,North York,"Downsview, North Park, Upwood Park"
150,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ..."
204,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St..."


In [7]:
postal_codes.shape

(103, 3)

In [8]:
list_of_longitudes = list()
list_of_latitudes = list()
for postal_code in postal_list:
    search = '{}, Toronto, Ontario'.format(postal_code)
    lat_lng_coords = None
    k_tries = 0
    
    while(lat_lng_coords is None and k_tries < 100 ):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        k_tries = k_tries + 1
    list_of_latitudes.append(lat_lng_coords[0])
    list_of_longitudes.append(lat_lng_coords[1])

In [9]:
postal_codes["latitude"] = list_of_latitudes
postal_codes["longitude"] = list_of_longitudes

In [10]:


import folium

m = folium.Map(location=[43.7 ,-79.3832],
    zoom_start=12)


for index, row in postal_codes.iterrows():
    label = '{}, {}'.format(row["Postcode"], row["Borough"])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row["latitude"], row["longitude"]],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(m)  


m



In [11]:
# The code was removed by Watson Studio for sharing.

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
postal_codes.head()

,Postcode,Borough,Neighborhoods,latitude,longitude
2,M3A,North York,Parkwoods,43.752420,-79.329242
3,M4A,North York,Victoria Village,43.730600,-79.313265
4,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
5,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723270,-79.451286
7,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715


In [14]:
LIMIT = 100
toronto_venues = getNearbyVenues(names=postal_codes['Neighborhoods'],
                                   latitudes=postal_codes['latitude'],
                                   longitudes=postal_codes['longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights, Lawrence Manor
Queen's Park
Not assigned
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The Danforth West, Riv

In [15]:

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")



In [16]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75242,-79.329242,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75242,-79.329242,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.75242,-79.329242,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Victoria Village,43.73060,-79.313265,Wigmore Park,43.731023,-79.310771,Park
4,Victoria Village,43.73060,-79.313265,Memories of Africa,43.726602,-79.312427,Grocery Store


In [17]:
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.03,0.0,0.01,0.0,0.03,0.0,...,0.0,0.01,0.0,0.0000,0.000000,0.01,0.0,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0000,0.066667,0.00,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0000,0.000000,0.00,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0625,0.000000,0.00,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0000,0.000000,0.00,0.0,0.0,0.0,0.0


In [19]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 5, 0, 8, 4, 0, 0, 6, 0], dtype=int32)

In [20]:
kmeans.labels_

array([0, 0, 5, 0, 8, 4, 0, 0, 6, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0,
       9, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 4, 0, 0, 6, 0, 6, 0, 0, 0, 0,
       6, 0, 4, 0, 0, 6, 8, 0, 0, 0, 0, 0, 6, 9, 0, 0, 0, 9, 0, 0, 2, 8,
       0, 6, 0, 0, 0, 0, 6, 0, 6, 1, 7, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 6,
       0, 8, 0, 6, 0, 0, 0, 9, 0, 0, 0, 8], dtype=int32)

In [21]:
kmeans_data = pd.DataFrame(toronto_grouped.Neighborhood)
kmeans_data.insert(0, 'Cluster Labels', kmeans.labels_)

In [22]:
postal_codes.rename(columns={"Neighborhoods":"Neighborhood"}, inplace=True)
postal_codes = postal_codes.join(kmeans_data.set_index('Neighborhood'), on='Neighborhood')

In [44]:
postal_codes.dropna(inplace=True)
postal_codes = postal_codes.astype({'Cluster Labels': 'int'})

In [25]:
# create map
map_clusters = folium.Map(location=[43.7 ,-79.3832],
    zoom_start=12)



In [39]:
# set color scheme for the clusters

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]



In [45]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(postal_codes['latitude'], postal_codes['longitude'], postal_codes['Neighborhood'], postal_codes['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters